# 3D Droplet Oscillation

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Timestepping;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [ ]:
//var myBatch = ExecutionQueues[0];
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ]


A switch for running the worksheet locally (since we have a folder structure for the data, which is not copied on Jenkins)

In [ ]:
bool onLocalMachine = false;

In [ ]:
//foreach(var s in wmg.Sessions)
//    s.Delete(true);

## Verification of Initial Value data

Initial values and parameters for the simulation (intial droplet shape, Ohnesorg number, initial velocity)
were specified by project partner (TU Graz, Group Prof. Brenn).
Details can be found in Document `setup.pdf`, to be found in the same directory as this worksheet.

First, it is verified that the initial values chosen here actually match the specification.

In [ ]:
MultidimensionalArray[] ReferenceData = new MultidimensionalArray[5];

Load reference data for 5 different cases; These files contain two columns, i.e. the azimuth angle and the respective droplet radius.

In [ ]:
for(int iCase = 0; iCase < 5; iCase++) {
    if(OnLocalMachine)    
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/surfaceDropCase{iCase + 1}.txt");
    else
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"surfaceDropCase{iCase + 1}.txt");
}

Analytical expressions for the reference data (see `setup.pdf`); this is to verify that the definition of Legendre
Functions resp. Polynomials in BoSSS actually matches the definition used by the TU Graz group.

In [ ]:
double case1Radius(double angle) {
   double radius = 0.966781 + 0.4*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius;
}

In [ ]:
double case2Radius(double angle) {
   double radius = 0.977143 + 0.4*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
   return radius;
}

In [ ]:
double case3Radius(double angle) {
   double radius = 0.981839 + 0.4*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
   return radius;
}

In [ ]:
double case4Radius(double angle) {
   double radius = 0.991848 + 0.2*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius;
}

In [ ]:
double case5Radius(double angle) {
   double radius = 0.999721 + 0.05*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
   return radius;
}

In [ ]:
Func<double,double>[] Case_i_Radius = new Func<double,double>[] { 
    case1Radius, case2Radius, case3Radius, case4Radius, case5Radius 
}; 

Conversion to cartesian coordinates in order to match the data and verification against analytical expression:

In [ ]:
double[][] refX = new double[5][];
double[][] refZ = new double[5][];
for(int iCase = 0; iCase < 5; iCase++) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    double[] radius = ReferenceData[iCase].GetColumn(1);
       
    double RadiusErrorNorm = 0.0;
    int I = angle.Length;
    double[] x1 = new double[I], z1 = new double[I];
    for(int i = 0; i < I; i++) {
        x1[i] = Math.Sin(angle[i])*radius[i];
        z1[i] = Math.Cos(angle[i])*radius[i];
        
        double radius_expr = Case_i_Radius[iCase](angle[i]);
        RadiusErrorNorm += (radius[i] - radius_expr).Pow2();
    }
    RadiusErrorNorm = RadiusErrorNorm.Sqrt();
    Console.WriteLine($"Comparison error for radius in case {iCase + 1}: {RadiusErrorNorm}");
    // Note: since the factors in `setup.pdf` are only provided up to 6 digits, an error threshold of 1e-5 seems reasonable.
    Assert.LessOrEqual(RadiusErrorNorm, 1e-5, "Error in comparing reference data against Legendre polynomials in BoSSS");
    
    refX[iCase] = x1;
    refZ[iCase] = z1;
}

Comparison error for radius in case 1: 8.451542560852182E-07
Comparison error for radius in case 2: 2.5354627647025936E-06
Comparison error for radius in case 3: 6.9956707656638065E-06
Comparison error for radius in case 4: 6.761234036477466E-06
Comparison error for radius in case 5: 8.39441090790297E-06


### Plot of Reference Data

In [ ]:
Plot(refX[0], refZ[0], "Ref-Case1", ".x blue", 
     refX[1], refZ[1], "Ref-Case2", ".o red",
     refX[2], refZ[2], "Ref-Case3", ".+ green",
     refX[3], refZ[3], "Ref-Case4", ".* magenta",
     refX[4], refZ[4], "Ref-Case5", ".^ grey")

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.5 
 
 
 
 
 -1 
 
 
 
 
 -0.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 
 
 
 
 Ref-Case1 
 
 
 Ref-Case1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ref-Case2 
 
 
 Ref-Case2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ref-Case3 
 
 
 Ref-Case3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ref-Case4 
 
 
 Ref-Case4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ref-Case5 
 
 
 Ref-Case5

### Matching of the Spherical Harmonics against the provided Data

In [ ]:
var Phi1Init = new Formula(
"Phi1",
false,
"using ilPSP.Utils; " + 
"double Phi1(double[] X) { " + 
"     " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.966781*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"}");

In [ ]:
var Phi2Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.977143*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");

In [ ]:
var Phi3Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.981839*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");

In [ ]:
var Phi4Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.991848*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.2*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");

In [ ]:
var Phi5Init = new Formula(
"Phi5",
false,
"using ilPSP.Utils; " + 
"double Phi5(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.999721*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.05*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");

In [ ]:
IBoundaryAndInitialData[] Phi_iCase = new IBoundaryAndInitialData[]  { Phi1Init, Phi2Init, Phi3Init, Phi4Init, Phi5Init};

In [ ]:
double x0 = refX[0][3];
double z0 = refZ[0][3];
var X0 = new Vector(x0, 0, z0);

In [ ]:
X0.ToString()

(0.04098108562248867|0|1.365626351969305)

In [ ]:
double rRef = ReferenceData[0][0,1];
double theta = ReferenceData[0][0,0];
double r = case1Radius(theta);
(theta, r, rRef, r - rRef)

Item1,Item2,Item3,Item4
0,1.366781,1.3667809523809522,4.761904781602766E-08


In [ ]:
for(int iCase = 0; iCase < 5; iCase++) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    //double[] xI = refX[iCase];   
    //double[] zI = refZ[iCase];
    int I = angle.Length;
    
    double PhiErr = 0;
    for(int i = 0; i < I; i++) {
        double radius_expr = Case_i_Radius[iCase](angle[i]);    
        double x1 = Math.Sin(angle[i])*radius_expr;
        double z1 = Math.Cos(angle[i])*radius_expr;
    
        PhiErr += Phi_iCase[iCase].Evaluate(new Vector(x1, 0, z1), 0.0).Abs();
    }
    Console.WriteLine($"Phi error for case {iCase}: {PhiErr}");
    Assert.LessOrEqual(PhiErr, 1e-10, "Level-Set function is not zero at desired surface.");
    
    Assert.IsTrue(Phi_iCase[iCase].Evaluate(new Vector(1e-5, 1e-5, 1e-5), 0.0) < 0, "Inside must be phase A/negative");
    Assert.IsTrue(Phi_iCase[iCase].Evaluate(new Vector(1e+1, 1e+1, 1e+1), 0.0) > 0, "Outside must be phase B/positive");
}

Phi error for case 0: 2.0650148258027912E-14
Phi error for case 1: 2.2315482794965646E-14
Phi error for case 2: 3.530509218307998E-14
Phi error for case 3: 1.532107773982716E-14
Phi error for case 4: 1.2545520178264269E-14


### Initial Velocities

In [ ]:
//var polVel = IMatrixExtensions.LoadFromTextFile($"polarVelCase1.txt");
//var radVel = IMatrixExtensions.LoadFromTextFile($"radialVelCase1.txt");

In [ ]:
var anaVel = new IBoundaryAndInitialData[5,3];

for(int iCase = 0; iCase < 5; iCase++) {
    MultidimensionalArray polVel;
    MultidimensionalArray radVel;
    if (OnLocalMachine) {
        polVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/polarVelCase{iCase + 1}.txt");
        radVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/radialVelCase{iCase + 1}.txt");
    } else {
        polVel = IMatrixExtensions.LoadFromTextFile($"polarVelCase{iCase + 1}.txt");
        radVel = IMatrixExtensions.LoadFromTextFile($"radialVelCase{iCase + 1}.txt");
    }
    Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(0), radVel.GetColumn(0)));
    Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(1), radVel.GetColumn(1)));
    
    double[] radiusS = polVel.GetColumn(0);
    double[] anglesS = polVel.GetColumn(1);
    double[] polVelS = polVel.GetColumn(2);
    double[] radVelS = radVel.GetColumn(2);
    
    var velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
    velX.SetData(anglesS, radiusS, polVelS, radVelS);
    var velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
    velY.SetData(anglesS, radiusS, polVelS, radVelS);
    var velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
    velZ.SetData(anglesS, radiusS, polVelS, radVelS);
    
    anaVel[iCase, 0] = velX;
    anaVel[iCase, 1] = velY;
    anaVel[iCase, 2] = velZ;
}

## Grid Creation

In [ ]:
//foreach(var g in BoSSSshell.WorkflowMgm.Grids)
//   g.Delete(true);

### Quater-Domain grids
(Symmetry planes at $x = 0$ and $y = 0$)

In [ ]:
Dictionary<string, IGridInfo[]> gridTypes = new Dictionary<string, IGridInfo[]>();

In [ ]:
int[] Resolutions = new int[] { 6 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
double scale = 1.0;
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_quarterDomain";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
        double[] yNodes = xNodes;
        double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
        var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Wall.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}
gridTypes.Add("wallBC", Grids);

Grid already found in database - identifid by name OscillatingDroplet3D_6x6x12_quarterDomain


In [ ]:
Resolutions = new int[] { 6 };
Grids = new IGridInfo[Resolutions.Length];
scale = 1.0;
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_quarterDomain";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
        double[] yNodes = xNodes;
        double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
        var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Pressure_Outlet.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}
gridTypes.Add("pressureOutletBC", Grids);

Grid already found in database - identifid by name OscillatingDroplet3D_6x6x12_quarterDomain


In [ ]:
//var g = (wmg.Grids[0] as GridProxy).RealGrid;

In [ ]:
//(g.iGridData as GridData).GlobalBoundingBox

In [ ]:
//wmg.Sessions[0].Delete(true);

## Setup of control objects for all solver runs

In [ ]:
string[] gridTypeKeys = new string[] { "wallBC", "pressureOutletBC" };
(int Case, double Ohnesorg)[] Cases = new[] { (1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.56) };
(double dt, int timesteps)[] Cases_time = new[] { (5e-3, 100), (2e-3, 250), (2e-3, 250), (5e-3, 100), (5e-3, 100)}; //{ (5e-3, 1400), (2e-3, 3500), (2e-3, 3500), (5e-3, 1400), (5e-3, 800)};
int[] AMRlevels = new int[] {0, 1, 1, 0, 0};

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();
int[] DegreeS = new int[] { 3 };
//bool[] useAMR = new bool[] { true };
bool[] useInitial = new bool[] { true, false };
bool[] useNewton = new bool[] { false };

// string grdKey = gridTypeKeys[1];
// Grids = gridTypes[grdKey];

//foreach(bool bAMR in useAMR) {
foreach(bool bInitial in useInitial) {
foreach(bool bNewton in useNewton) {
foreach(int k in DegreeS) {
foreach(string grdKey in gridTypeKeys) {
foreach(var grd in gridTypes[grdKey]) {
foreach(var myCase in Cases) {
    long J = grd.NumberOfCells;
    int AMRlvl = AMRlevels[myCase.Case-1];
    string JobName = $"OD3D_J{J}k{k}_{grdKey}_amr{AMRlvl}_case{myCase.Case}_Oh{myCase.Ohnesorg}_StartUp";
    if(bInitial) {
        JobName = JobName + "_AnalytInitPhaseAonly";
    }
    if(bNewton) {
        JobName = JobName + "_Newton";
    }
    Console.WriteLine("Case: " + JobName);

    var C = new XNSE_Control();
    
    C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    
    C.InitialValues.Add("Phi", Phi_iCase[myCase.Case - 1]);
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.rho_B = 0.001;
    C.PhysicalParameters.mu_A = myCase.Ohnesorg;
    C.PhysicalParameters.mu_B = myCase.Ohnesorg/1000;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 1;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.StokesExtension;
    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3;
    if(bNewton) {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    } else {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
    }
    C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.Timestepper_BDFinit = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.dtFixed = Cases_time[myCase.Case - 1].dt;
    C.NoOfTimesteps = Cases_time[myCase.Case - 1].timesteps;
    
    if(AMRlvl > 0) {
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = AMRlvl }
        );
    }
    
    if(bInitial) {
        C.AddInitialValue("VelocityX#A", anaVel[myCase.Case - 1, 0]);
        //C.AddInitialValue("VelocityX#B", anaVel[myCase.Case - 1, 0]);
        C.AddInitialValue("VelocityY#A", anaVel[myCase.Case - 1, 1]);
        //C.AddInitialValue("VelocityY#B", anaVel[myCase.Case - 1, 1]);
        C.AddInitialValue("VelocityZ#A", anaVel[myCase.Case - 1, 2]);
        //C.AddInitialValue("VelocityZ#B", anaVel[myCase.Case - 1, 2]);
    }
    
    C.PostprocessingModules.Add(new SphericalHarmonicsLogging() { MaxL = 8, RotSymmetric = true });
    C.PostprocessingModules.Add(new DropletMetricsLogging() { AxisSymmetric = true });
    C.PostprocessingModules.Add(new EnergyLogging());
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}
}
}
}
}

Case: OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_wallBC_amr1_case3_Oh0.1_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_wallBC_amr0_case4_Oh0.1_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_wallBC_amr0_case5_Oh0.56_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_pressureOutletBC_amr0_case1_Oh0.1_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_pressureOutletBC_amr1_case2_Oh0.1_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_pressureOutletBC_amr1_case3_Oh0.1_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_pressureOutletBC_amr0_case4_Oh0.1_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_pressureOutletBC_amr0_case5_Oh0.56_StartUp_AnalytInitPhaseAonly
Case: OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp
Case: OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp
Case: OD3D_J432k3_wallBC_amr1_case3_Oh0.1_StartUp
Case: OD3D_J432k3_wallBC_amr0_case4_Oh0.1_StartUp
Case: OD3D_J432k3_wallBC_amr0_case5_Oh

In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal");
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal");
    }
}

## Launch Jobs

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,OD3D_J432k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly
1,OD3D_J3456k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly


In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Deploying job OD3D_J432k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly ... 
Deploying executables and additional files ...
Deployment directory: D:\local\binaries\OscillatingDroplet3D-XNSE_Solver2022Mar16_133801
copied 50 files.
   written file: control.obj
deployment finished.
Starting mini batch processor in external process...
Started mini batch processor on local machine, process id is 26296.
started.

Deploying job OD3D_J3456k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly ... 
Deploying executables and additional files ...
Deployment directory: D:\local\binaries\OscillatingDroplet3D-XNSE_Solver2022Mar16_133805
copied 50 files.
   written file: control.obj
deployment finished.
Loading session 0120c987-faf5-46cd-857c-5c2b365d8e30 failed with message 'Could not find file 'D:\local\OscillatingDroplet3D\sessions\0120c987-faf5-46cd-857c-5c2b365d8e30\Session.info'.'
Mini batch processor is already running.
Loading session 0120c987-faf5-46cd-857c-5c2b365d8e30 failed wi

In [ ]:
wmg.AllJobs

Loading session a4a8b9b2-27ab-4fbe-81e5-60a616bff835 failed with message 'Could not find file 'D:\local\OscillatingDroplet3D\sessions\a4a8b9b2-27ab-4fbe-81e5-60a616bff835\Session.info'.'


#0: OD3D_J432k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly: InProgress (MiniBatchProcessor client  LocalPC @D:\local\binaries)	OD3D_J432k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly: InProgress (MiniBatchProcessor client  LocalPC @D:\local\binaries)
#1: OD3D_J3456k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly: InProgress (MiniBatchProcessor client  LocalPC @D:\local\binaries)	OD3D_J3456k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly: InProgress (MiniBatchProcessor client  LocalPC @D:\local\binaries)


In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

Timeout.


In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

Starting external console ...
(You may close the new window at any time, the job will continue.)


In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()